# TCG Shadowbox Generator - クイックスタート

このノートブックでは、TCGカードのイラストを深度推定とクラスタリングで階層化し、
インタラクティブな3Dシャドーボックスとして表示する方法を説明します。

## 目次
1. セットアップ
2. 画像の読み込み
3. イラスト領域の特定
4. 深度推定とクラスタリング
5. 2D可視化
6. 3Dシャドーボックス表示

## 1. セットアップ

必要なモジュールをインポートします。

In [ ]:
# 基本的なインポート
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# shadowboxモジュールのインポート
from shadowbox import create_pipeline
from shadowbox.config import BoundingBox, ShadowboxSettings
from shadowbox.detection import detect_illustration_region
from shadowbox.gui import ImageSelector, select_illustration_region, load_from_url
from shadowbox.visualization import (
    create_depth_heatmap,
    show_clustering_summary,
    render_shadowbox,
    RenderOptions,
)

# matplotlibの日本語フォント設定（Windows）
import matplotlib
matplotlib.rcParams['font.family'] = ['MS Gothic', 'Yu Gothic', 'Meiryo', 'sans-serif']
matplotlib.rcParams['axes.unicode_minus'] = False  # マイナス記号の文字化け対策

print("セットアップ完了！")

## 2. 画像の読み込み

画像を読み込む方法は3つあります：
1. URLから読み込み
2. ローカルファイルから読み込み
3. ディレクトリからギャラリー選択

### 方法1: URLから読み込み

In [ ]:
# URLから画像を読み込む場合
# 実際のカード画像URLに変更してください
image_url = "https://example.com/your-card-image.png"  # ← ここにURLを設定
image = load_from_url(image_url)

# または、ローカルファイルから読み込み
# from shadowbox.utils.image import load_image
# image = load_image("path/to/your/card.png")

# デモ用: ダミー画像を作成（実際の画像を使う場合はコメントアウト）
# image = Image.new("RGB", (250, 350), color=(50, 50, 50))
# pixels = image.load()
# for y in range(40, 240):
#     for x in range(20, 230):
#         r = (x * 3 + y) % 256
#         g = (x + y * 2) % 256
#         b = (x * 2 + y * 3) % 256
#         pixels[x, y] = (r, g, b)

# 画像を表示
plt.figure(figsize=(6, 8))
plt.imshow(image)
plt.title("読み込んだ画像")
plt.axis("off")
plt.show()

### 方法2: ローカルファイルから読み込み

In [ ]:
# ローカルファイルから読み込む場合
from shadowbox.utils.image import load_image
image = load_image("../data/cards/your-card-image.png")

# 画像を表示
plt.figure(figsize=(6, 8))
plt.imshow(image)
plt.title("読み込んだ画像")
plt.axis("off")
plt.show()

### 方法3: ディレクトリからギャラリー選択

In [ ]:
# ディレクトリ内の画像をギャラリー表示して選択
selector = ImageSelector("../data/cards/")
selector.show_gallery()  # サムネイル付きで一覧表示
image = selector.select(0)  # インデックスで選択

## 3. イラスト領域の特定

カード画像からイラスト部分を特定します。
自動検出と手動選択の2つの方法があります。

### 方法A: 自動検出

自動検出では10種類の手法を並行実行し、最も信頼度の高い結果を返します：
- `edge_detection`: Cannyエッジ検出 + 輪郭検出
- `hsv_threshold`: HSV彩度・明度の大津閾値処理
- `contour_detection`: 適応的閾値 + 4角形近似
- `grid_scoring`: グリッド分割 + スライディングウィンドウ
- `boundary_contrast`: 内部複雑度 + 境界単色度
- `frame_analysis`: フレーム色から境界を探索（外→内）
- `band_complexity`: 水平帯の複雑度分析
- `horizontal_lines`: Hough Transform で水平線検出
- `center_expansion`: 中央から複雑度で拡張
- `gradient_richness`: **勾配の豊かさで境界検出** (NEW!)

💡 `gradient_richness` はモノクロ画像の微分を計算し、勾配変化の豊かさ（分散）が外れ値以下に下がる地点を境界とします。

In [ ]:
# 自動検出でイラスト領域を特定
detection_result = detect_illustration_region(image)

print(f"検出方法: {detection_result.method}")
print(f"信頼度: {detection_result.confidence:.2f}")
print(f"バウンディングボックス: {detection_result.bbox}")

# 検出結果を可視化
fig, ax = plt.subplots(figsize=(8, 10))
ax.imshow(image)

bbox = detection_result.bbox
rect = plt.Rectangle(
    (bbox.x, bbox.y), bbox.width, bbox.height,
    linewidth=2, edgecolor='lime', facecolor='none'
)
ax.add_patch(rect)
ax.set_title(f"自動検出結果 (信頼度: {detection_result.confidence:.2f})")
ax.axis("off")
plt.show()

#### A.1 パラメータをカスタマイズして検出

In [ ]:
from shadowbox.detection import RegionDetector

# パラメータをカスタマイズした検出器を作成
detector = RegionDetector(
    min_area_ratio=0.15,  # 最小面積比（カード全体の15%以上）
    max_area_ratio=0.70,  # 最大面積比（カード全体の85%以下）
    canny_low=30,         # Cannyエッジ検出の低閾値（小さいほど多くのエッジを検出）
    canny_high=100,       # Cannyエッジ検出の高閾値
)

# 検出実行
result_custom = detector.detect(image)

print(f"検出方法: {result_custom.method}")
print(f"信頼度: {result_custom.confidence:.2f}")
print(f"領域: x={result_custom.bbox.x}, y={result_custom.bbox.y}, "
      f"w={result_custom.bbox.width}, h={result_custom.bbox.height}")

In [ ]:
# A.1 カスタム検出結果を可視化
fig, ax = plt.subplots(figsize=(8, 10))
ax.imshow(image)

bbox_custom = result_custom.bbox
rect = plt.Rectangle(
    (bbox_custom.x, bbox_custom.y), bbox_custom.width, bbox_custom.height,
    linewidth=2, edgecolor='cyan', facecolor='none', linestyle='--'
)
ax.add_patch(rect)
ax.set_title(f"カスタム検出結果: {result_custom.method}\n信頼度: {result_custom.confidence:.2f}")
ax.axis("off")
plt.show()

#### A.2.1 特定の検出ロジックを直接適用

使用したい検出ロジックを直接指定して適用します。

**利用可能なメソッド:**
- `_detect_by_edges`: エッジ検出
- `_detect_by_hsv_threshold`: HSV閾値
- `_detect_by_contours`: 輪郭検出
- `_detect_by_grid_scoring`: グリッドスコアリング
- `_detect_by_boundary_contrast`: 境界コントラスト
- `_detect_by_frame_analysis`: フレーム解析
- `_detect_by_band_complexity`: 水平帯複雑度
- `_detect_by_horizontal_lines`: 水平線検出
- `_detect_by_center_expansion`: 中央拡張
- `_detect_by_gradient_richness`: 勾配の豊かさ ← おすすめ

In [ ]:
# 特定の検出ロジックを直接適用
from shadowbox.detection import RegionDetector

# 検出器を作成
direct_detector = RegionDetector(
    min_area_ratio=0.05,
    max_area_ratio=0.95,
)

# 使用するメソッドを選択（下のコメントを参考に変更）
# "_detect_by_gradient_richness"  ← おすすめ
# "_detect_by_edges"
# "_detect_by_hsv_threshold"
# "_detect_by_contours"
# "_detect_by_grid_scoring"
# "_detect_by_boundary_contrast"
# "_detect_by_frame_analysis"
# "_detect_by_band_complexity"
# "_detect_by_horizontal_lines"
# "_detect_by_center_expansion"

method_name = "_detect_by_gradient_richness"  # ← ここを変更

# 検出実行
cv_image = direct_detector._pil_to_cv(image)
detect_method = getattr(direct_detector, method_name)
direct_result = detect_method(cv_image)

if direct_result:
    print(f"検出方法: {direct_result.method}")
    print(f"信頼度: {direct_result.confidence:.2f}")
    print(f"領域: x={direct_result.bbox.x}, y={direct_result.bbox.y}, "
          f"w={direct_result.bbox.width}, h={direct_result.bbox.height}")
    
    # 結果を可視化
    fig, axes = plt.subplots(1, 2, figsize=(14, 8))
    
    # 検出領域
    axes[0].imshow(image)
    rect = plt.Rectangle(
        (direct_result.bbox.x, direct_result.bbox.y),
        direct_result.bbox.width, direct_result.bbox.height,
        linewidth=3, edgecolor='lime', facecolor='none'
    )
    axes[0].add_patch(rect)
    axes[0].set_title(f"{direct_result.method}\n信頼度: {direct_result.confidence:.2f}")
    axes[0].axis("off")
    
    # クロップ領域プレビュー
    cropped = image.crop((
        direct_result.bbox.x, direct_result.bbox.y,
        direct_result.bbox.x + direct_result.bbox.width,
        direct_result.bbox.y + direct_result.bbox.height
    ))
    axes[1].imshow(cropped)
    axes[1].set_title(f"クロップ領域\n({direct_result.bbox.width} x {direct_result.bbox.height} px)")
    axes[1].axis("off")
    
    plt.tight_layout()
    plt.show()
    
    # 以降の処理で使用するbboxを設定
    bbox = direct_result.bbox
    selected_result = direct_result
else:
    print(f"{method_name} では検出できませんでした")

#### A.2.2 各検出方法を個別にテストして比較

10種類の検出方法を個別に実行して結果を比較します：
- **赤**: エッジ検出 (edge_detection)
- **青**: HSV閾値 (hsv_threshold)
- **緑**: 輪郭検出 (contour_detection)
- **オレンジ**: グリッドスコアリング (grid_scoring)
- **マゼンタ**: 境界コントラスト (boundary_contrast)
- **シアン**: フレーム解析 (frame_analysis)
- **黄**: 水平帯複雑度 (band_complexity)
- **白**: 水平線検出 (horizontal_lines)
- **ライム**: 中央拡張 (center_expansion)
- **ピンク**: 勾配の豊かさ (gradient_richness) ← モノクロ微分の分散で境界検出

In [ ]:
# 各検出方法を個別にテスト（デバッグ用）
# 面積制約を緩めた検出器を作成
debug_detector = RegionDetector(
    min_area_ratio=0.05,  # より小さい面積も許容
    max_area_ratio=0.95,  # より大きい面積も許容
)

# 内部メソッドを直接呼び出して確認
cv_image = debug_detector._pil_to_cv(image)

edge_result = debug_detector._detect_by_edges(cv_image)
hsv_result = debug_detector._detect_by_hsv_threshold(cv_image)
contour_result = debug_detector._detect_by_contours(cv_image)
grid_result = debug_detector._detect_by_grid_scoring(cv_image)
boundary_result = debug_detector._detect_by_boundary_contrast(cv_image)
frame_result = debug_detector._detect_by_frame_analysis(cv_image)
band_result = debug_detector._detect_by_band_complexity(cv_image)
hlines_result = debug_detector._detect_by_horizontal_lines(cv_image)
center_result = debug_detector._detect_by_center_expansion(cv_image)
gradient_result = debug_detector._detect_by_gradient_richness(cv_image)

print("=== 各検出方法の結果 ===")
methods = [
    ("エッジ検出", edge_result),
    ("HSV閾値", hsv_result),
    ("輪郭検出", contour_result),
    ("グリッド", grid_result),
    ("境界コントラスト", boundary_result),
    ("フレーム解析", frame_result),
    ("水平帯複雑度", band_result),
    ("水平線検出", hlines_result),
    ("中央拡張", center_result),
    ("勾配の豊かさ", gradient_result),
]

for name, result in methods:
    if result:
        print(f"{name:12s}: {result.method:20s} 信頼度: {result.confidence:.2f}")
    else:
        print(f"{name:12s}: None")

# 検出できた結果を可視化
results_with_colors = [
    (edge_result, 'red'), 
    (hsv_result, 'blue'), 
    (contour_result, 'green'),
    (grid_result, 'orange'),
    (boundary_result, 'magenta'),
    (frame_result, 'cyan'),
    (band_result, 'yellow'),
    (hlines_result, 'white'),
    (center_result, 'lime'),
    (gradient_result, 'hotpink'),
]
results = [(r, c) for r, c in results_with_colors if r is not None]

if results:
    fig, ax = plt.subplots(figsize=(10, 12))
    ax.imshow(image)
    for r, color in results:
        rect = plt.Rectangle(
            (r.bbox.x, r.bbox.y), r.bbox.width, r.bbox.height,
            linewidth=2, edgecolor=color, facecolor='none',
            label=f"{r.method} ({r.confidence:.2f})"
        )
        ax.add_patch(rect)
    ax.legend(loc='upper right', fontsize=8)
    ax.set_title("各検出方法の結果比較")
    ax.axis("off")
    plt.show()
else:
    print("どの方法でも検出できませんでした")

In [ ]:
# 複数の候補を取得（信頼度順）
# ※ debug_detectorを使用（面積制約が緩い）
candidates = debug_detector.detect_with_candidates(image, max_candidates=10)

print("検出候補一覧:")
print("-" * 70)

if not candidates:
    print("候補が見つかりませんでした。面積比やCanny閾値を調整してください。")
else:
    for i, r in enumerate(candidates):
        print(f"[{i}] {r.method:20s} 信頼度: {r.confidence:.2f}  "
              f"({r.bbox.x}, {r.bbox.y}, {r.bbox.width}x{r.bbox.height})")

    # 候補を可視化（2行で表示）
    num_candidates = len(candidates)
    num_cols = min(num_candidates, 5)
    num_rows = (num_candidates + num_cols - 1) // num_cols
    
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(4 * num_cols, 5 * num_rows))
    if num_rows == 1:
        axes = [axes] if num_cols == 1 else axes
    else:
        axes = axes.flatten()

    colors = ['lime', 'cyan', 'magenta', 'yellow', 'orange', 'red', 'blue', 'green', 'white', 'hotpink']
    
    for i, r in enumerate(candidates):
        ax = axes[i] if num_candidates > 1 else axes[0]
        ax.imshow(image)
        rect = plt.Rectangle(
            (r.bbox.x, r.bbox.y), r.bbox.width, r.bbox.height,
            linewidth=2, edgecolor=colors[i % len(colors)], facecolor='none'
        )
        ax.add_patch(rect)
        ax.set_title(f"[{i}] {r.method}\n信頼度: {r.confidence:.2f}", fontsize=9)
        ax.axis("off")

    # 余分なaxesを非表示
    for i in range(num_candidates, len(axes) if isinstance(axes, list) or isinstance(axes, np.ndarray) else 1):
        if isinstance(axes, (list, np.ndarray)) and i < len(axes):
            axes[i].axis("off")
            axes[i].set_visible(False)

    plt.suptitle("検出候補の比較（信頼度順）")
    plt.tight_layout()
    plt.show()

In [ ]:
# 候補から選択（インデックスを変更して好みの結果を選ぶ）
if not candidates:
    print("候補がありません。手動選択（方法B）を使用するか、検出パラメータを調整してください。")
    # デフォルトのbboxを使用
    bbox = BoundingBox(
        x=int(image.width * 0.1),
        y=int(image.height * 0.1),
        width=int(image.width * 0.8),
        height=int(image.height * 0.8),
    )
    selected_result = None
    selected_index = -1
else:
    selected_index = 3  # ← ここを変更して別の候補を選択
    selected_result = candidates[selected_index]
    
    print(f"選択した候補: [{selected_index}] {selected_result.method}")
    print(f"バウンディングボックス: {selected_result.bbox}")
    
    # 以降の処理で使用するbboxを設定
    bbox = selected_result.bbox

In [ ]:
# 選択した候補を可視化（クロップ領域のプレビュー付き）
fig, axes = plt.subplots(1, 2, figsize=(14, 8))

# 元画像と検出領域
axes[0].imshow(image)
rect = plt.Rectangle(
    (bbox.x, bbox.y), bbox.width, bbox.height,
    linewidth=3, edgecolor='lime', facecolor='none'
)
axes[0].add_patch(rect)

if selected_result:
    axes[0].set_title(f"選択した領域: [{selected_index}] {selected_result.method}\n信頼度: {selected_result.confidence:.2f}")
else:
    axes[0].set_title("デフォルト領域（自動検出失敗時のフォールバック）")
axes[0].axis("off")

# クロップされた領域のプレビュー
cropped_preview = image.crop((bbox.x, bbox.y, bbox.x + bbox.width, bbox.y + bbox.height))
axes[1].imshow(cropped_preview)
axes[1].set_title(f"イラスト領域プレビュー\n({bbox.width} x {bbox.height} px)")
axes[1].axis("off")

plt.tight_layout()
plt.show()

### 方法B: 手動選択（インタラクティブ）

Jupyter Notebook内でインタラクティブに領域を選択します。

**手順:**
1. 下のセルを実行 → 画像が表示される
2. **左クリック1回目**: 矩形の1点目を指定（赤い＋マーカー）
3. **左クリック2回目**: 矩形の2点目を指定 → 緑の矩形が表示
4. **右クリック**: リセット（やり直し）
5. 次のセルを実行して結果を取得

**注意:** `%matplotlib widget` と `ipympl` が必要です（`uv sync --extra jupyter` でインストール）。

In [ ]:
# Jupyter用インタラクティブバックエンドに切り替え
%matplotlib widget

from shadowbox.gui import JupyterRegionSelector

# セレクターを作成して表示
selector = JupyterRegionSelector(image)
selector.show()

# ↓ 選択完了後、次のセルで実行して結果を取得

In [ ]:
# 選択結果を取得（上のセルで選択完了後に実行）
bbox = selector.get_bbox()
if bbox:
    print(f"選択された領域: {bbox}")
    
    # プレビュー表示
    cropped = selector.get_cropped()
    plt.figure(figsize=(8, 8))
    plt.imshow(cropped)
    plt.title(f"選択領域 ({bbox.width} x {bbox.height})")
    plt.axis("off")
    plt.show()
else:
    print("領域が選択されていません。上のセルで左クリック2回で選択してください。")

## 4. 深度推定とクラスタリング

パイプラインを使用して、画像から深度を推定し、
レイヤーに分割します。

In [ ]:
# パイプラインを作成（初回実行時はモデルのダウンロードが行われます）
# use_mock_depth=True はテスト用（実際の深度推定を使用する場合はFalseに）
pipeline = create_pipeline(use_mock_depth=True)

print("パイプライン作成完了")

In [ ]:
# パイプラインを実行
# k: レイヤー数（指定しない場合は最適な値を自動探索）
# include_frame: フレーム（枠）を含めるかどうか
result = pipeline.process(
    image,
    custom_bbox=bbox,
    k=5,  # 5つのレイヤーに分割
    include_frame=True,
)

print(f"最適なレイヤー数: {result.optimal_k}")
print(f"クロップ後のサイズ: {result.cropped_image.shape}")
print(f"メッシュのレイヤー数: {result.mesh.num_layers}")
print(f"総頂点数: {result.mesh.total_vertices}")

## 5. 2D可視化

深度マップとレイヤー分割の結果を2Dで可視化します。

### 5.1 深度ヒートマップ

In [ ]:
# 深度マップをヒートマップで表示
fig, axes = create_depth_heatmap(
    result.depth_map,
    result.cropped_image,
    cmap="viridis",
    title="深度推定結果",
)
plt.show()

### 5.2 クラスタリングサマリー

In [ ]:
# クラスタリング結果の総合サマリーを表示
fig, axes = show_clustering_summary(
    result.cropped_image,
    result.depth_map,
    result.labels,
    result.centroids,
)
plt.show()

## 6. 3Dシャドーボックス表示

Vedoを使用してインタラクティブな3D表示を行います。

**操作方法:**
- マウス左ドラッグ: 回転
- マウス右ドラッグ: パン
- スクロール: ズーム

In [ ]:
# レンダリングオプションを設定
options = RenderOptions(
    background_color=(30, 30, 40),  # 暗い背景
    point_size=4.0,  # ポイントサイズ
    show_frame=True,  # フレームを表示
    window_size=(1000, 800),
    title="TCG Shadowbox Viewer",
)

# 3Dレンダリング（新しいウィンドウが開きます）
# 注: Jupyter環境では別ウィンドウで表示されます
render_shadowbox(result.mesh, options)

### 6.1 レイヤー分解表示（デバッグ用）

In [ ]:
# レイヤーを分離して表示（構造を確認しやすくする）
from shadowbox.visualization import render_layers_exploded

render_layers_exploded(result.mesh, gap_multiplier=3.0, options=options)

## 補足: 実際の深度推定を使用する場合

モック深度推定ではなく、実際のDepth Anything v2モデルを使用する場合は、
`use_mock_depth=False`を指定してください。

**注意**: 初回実行時はモデルのダウンロードに時間がかかります（約500MB）。

In [ ]:
# 実際の深度推定を使用する場合（コメントを解除）
# pipeline_real = create_pipeline(use_mock_depth=False)
# result_real = pipeline_real.process(image, custom_bbox=bbox, k=5)

## カスタム設定の使用

In [ ]:
# カスタム設定でパイプラインを作成
settings = ShadowboxSettings()

# クラスタリング設定
settings.clustering.min_k = 3
settings.clustering.max_k = 8

# レンダリング設定
settings.render.layer_thickness = 0.15
settings.render.layer_gap = 0.02

# カスタム設定でパイプラインを作成
# pipeline_custom = create_pipeline(settings, use_mock_depth=True)

print("カスタム設定:")
print(f"  クラスタリング: k={settings.clustering.min_k}-{settings.clustering.max_k}")
print(f"  レイヤー厚み: {settings.render.layer_thickness}")

## 終わりに

このノートブックでは、TCGカードからインタラクティブな3Dシャドーボックスを
作成する基本的なワークフローを紹介しました。

### 次のステップ
- カード画像のURLを変更して、自分のカードで試してみてください
- レイヤー数（k）を変更して、最適な見た目を探してみてください
- レンダリングオプションを調整して、好みの表示にカスタマイズしてください

### 参考リンク
- [GitHub Repository](https://github.com/2ROSSO/shadowbox-generator)
- [Depth Anything v2](https://huggingface.co/depth-anything)